# Queda Livre — Colab-ready

Este notebook gera um dataset sintético de queda livre (tempo → posição), treina uma rede neural simples para estimar a posição a partir do tempo, e salva os artefatos necessários para inferência.

Siga a ordem: `Setup` → `Gerar dados` → `Treinar` → `Exportar SavedModel (opcional)`


In [ ]:
# Setup para Colab / Jupyter
# - Detecta se estamos no Google Colab
# - (Opcional) monta o Drive se desejar persistir arquivos
# - Garante pastas `data/` e `model/`
# - Não força instalação do TensorFlow (Colab já possui TF); instala apenas pacotes leves se estiverem ausentes
import os
import sys
import importlib
import subprocess

IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

print('Google Colab:', IN_COLAB)

# Se quiser persistir artefatos automaticamente no Drive, mude para True
MOUNT_DRIVE = False
if IN_COLAB and MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive montado em /content/drive')

# Instala pacotes leves se faltarem (não instala tensorflow)
def pip_install(pkg):
    print(f'Instalando {pkg}...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

for pkg in ('pandas', 'scikit-learn'):
    if importlib.util.find_spec(pkg) is None:
        pip_install(pkg)
    else:
        print(f'{pkg} já presente')

# Garantir diretórios
os.makedirs('data', exist_ok=True)
os.makedirs('model', exist_ok=True)

# Seeds previsíveis para demonstração
try:
    import tensorflow as tf
    tf.random.set_seed(42)
except Exception:
    pass

import numpy as np
np.random.seed(42)

print('\nSetup concluído. Execute as células abaixo (geração de dados → treino → export) na ordem.')


### 1) Preparação do ambiente

Propósito: detectar se estamos executando no Google Colab e preparar o ambiente de forma segura, sem reinstalar pacotes críticos (TensorFlow/numpy) que podem causar conflitos no runtime.

### 2) Preparação rápida do dataset e parâmetros

Nesta seção geramos os dados sintéticos que serão usados no treino e nos testes. Você pode ajustar `noise_std` para simular medições ruidosas ou substituir a função analítica por um integrador numérico para incluir arrasto.

In [ ]:
# 2) Gerar dataset (t -> s)
import numpy as np

def free_fall_distance(t, g=9.81, v0=0.0, s0=0.0):
    return s0 + v0 * t + 0.5 * g * (t ** 2)


def generate_dataset(n_samples=5000, t_min=0.0, t_max=10.0, noise_std=0.0, seed=42):
    rng = np.random.default_rng(seed)
    t = rng.uniform(t_min, t_max, size=(n_samples, 1)).astype(np.float32)
    s = free_fall_distance(t).astype(np.float32)
    if noise_std > 0:
        s += rng.normal(0, noise_std, size=s.shape).astype(np.float32)
    return t, s


t_train, s_train = generate_dataset(5000, 0.0, 10.0, noise_std=0.0, seed=1)
 t_test, s_test = generate_dataset(1000, 0.0, 10.0, noise_std=0.0, seed=2)

import os
os.makedirs('data', exist_ok=True)
np.savez_compressed('data/dataset.npz', t_train=t_train, s_train=s_train, t_test=t_test, s_test=s_test)
print('Dataset salvo em data/dataset.npz — exemplos:')
print('t[0]=', t_train[0,0], 's[0]=', s_train[0,0])


Gerar arquivo de inferência (CSV/NPZ):

Esta célula cria um arquivo `data/times_for_infer.csv` contendo uma coluna `t` com 100 valores uniformes entre 0 e 10s. Também salva `data/dataset_infer.npz` com `t` e `s_true` para referência/validação.

In [ ]:
# Cria arquivos para inferência em lote
import numpy as np
import os
os.makedirs('data', exist_ok=True)
# 100 tempos uniformes entre 0 e 10s
t_infer = np.linspace(0.0, 10.0, num=100, dtype=np.float32).reshape(-1,1)
# calcula a posição verdadeira sem ruído para referência
def free_fall_distance(t, g=9.81, v0=0.0, s0=0.0):
    return s0 + v0 * t + 0.5 * g * (t ** 2)
s_true = free_fall_distance(t_infer)
# salvar CSV com coluna 't' (compatível com QuedaLivre_infer_colab upload CSV)
import pandas as pd
df = pd.DataFrame({'t': t_infer.flatten()})
csv_path = 'data/times_for_infer.csv'
df.to_csv(csv_path, index=False)
# salvar npz com t e s_true
np.savez_compressed('data/dataset_infer.npz', t=t_infer, s_true=s_true)
print('Arquivos gerados:')
print(' -', csv_path)
print(' - data/dataset_infer.npz')


### 3) Treinamento do modelo

A célula abaixo carrega `data/dataset.npz`, normaliza os dados, monta uma rede rasa e treina. Ao final salva `model/fall_model.keras` e `model/scaler.npz`.

In [ ]:
# Célula adicional — cria arquivos para inferência em lote
import numpy as np
import os
os.makedirs('data', exist_ok=True)
# 100 tempos uniformes entre 0 e 10s
t_infer = np.linspace(0.0, 10.0, num=100, dtype=np.float32).reshape(-1,1)
# calcula a posição verdadeira sem ruído para referência
def free_fall_distance(t, g=9.81, v0=0.0, s0=0.0):
    return s0 + v0 * t + 0.5 * g * (t ** 2)
s_true = free_fall_distance(t_infer)
# salvar CSV com coluna 't' (compatível com QuedaLivre_infer_colab upload CSV)
import pandas as pd
df = pd.DataFrame({'t': t_infer.flatten()})
csv_path = 'data/times_for_infer.csv'
df.to_csv(csv_path, index=False)
# salvar npz com t e s_true
np.savez_compressed('data/dataset_infer.npz', t=t_infer, s_true=s_true)
print('Arquivos gerados:')
print(' -', csv_path)
print(' - data/dataset_infer.npz')


### 2) Geração do dataset

Propósito: gerar um dataset sintético mapeando tempo `t` para posição `s(t)` usando a fórmula da queda livre. Suporta adicionar ruído nas medições para simular imperfeições.

Entradas: parâmetros de geração — número de amostras, intervalo de tempo (t_min, t_max), desvio padrão do ruído, semente RNG.

Saídas: arrays `t_train`, `s_train`, `t_test`, `s_test`, e arquivo salvo `data/dataset.npz`.

Dicas:
- Use `seed` para reprodutibilidade.
- Se quiser incluir arrasto, substitua a função analítica por um integrador numérico (ex.: RK4) e re-gere o dataset.


In [ ]:
# Treinamento do modelo
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

data = np.load('data/dataset.npz')
t_train = data['t_train']
s_train = data['s_train']
t_test = data['t_test']
s_test = data['s_test']

# normalização simples
t_mean, t_std = float(t_train.mean()), float(t_train.std())
s_mean, s_std = float(s_train.mean()), float(s_train.std())
t_train_n = (t_train - t_mean) / t_std
t_test_n = (t_test - t_mean) / t_std
s_train_n = (s_train - s_mean) / s_std
s_test_n = (s_test - s_mean) / s_std

def build_model():
    model = models.Sequential([
        layers.Input(shape=(1,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
    return model

model = build_model()
model.summary()
history = model.fit(t_train_n, s_train_n, epochs=60, batch_size=64, validation_split=0.1, verbose=2)

# avaliar
test_loss, test_mae = model.evaluate(t_test_n, s_test_n, verbose=0)
print(f'Teste - MSE: {test_loss:.6f}, MAE: {test_mae:.6f}')

os.makedirs('model', exist_ok=True)
model.save('model/fall_model.keras')
# salve scalers como floats para evitar problemas no carregamento
np.savez_compressed('model/scaler.npz', t_mean=t_mean, t_std=t_std, s_mean=s_mean, s_std=s_std)
print('Modelo e scalers salvos em model/')


### 4) Inferência e uso do modelo

Esta seção mostra como carregar o modelo salvo e realizar predições (ex.: carregar `data/times_for_infer.csv`).

In [ ]:
# Exemplo de inferência (carregando .keras e scalers)
import numpy as np
from tensorflow.keras.models import load_model

# procura por caminhos possíveis
candidates = ['model/fall_model.keras', 'model/fall_model.h5', 'model/fall_model']
model_path = None
import os
for c in candidates:
    if os.path.exists(c):
        model_path = c
        break

if model_path is None:
    print('Nenhum modelo encontrado em model/. Rode a célula de treino primeiro.')
else:
    print('Carregando modelo de', model_path)
    model = load_model(model_path)
    scal = np.load('model/scaler.npz')
    # carregar como floats
    t_mean = float(scal['t_mean'])
    t_std = float(scal['t_std'])
    s_mean = float(scal['s_mean'])
    s_std = float(scal['s_std'])

    # função de predição rápida
    def predict_time(t_values):
        import numpy as np
        t_arr = np.array(t_values, dtype=np.float32).reshape(-1,1)
        t_n = (t_arr - t_mean) / t_std
        s_n = model.predict(t_n, verbose=0)
        s = s_n * s_std + s_mean
        return s.flatten()

    # exemplo: carregar CSV gerado anteriormente
    import pandas as pd
    if os.path.exists('data/times_for_infer.csv'):
        df = pd.read_csv('data/times_for_infer.csv')
        if 't' in df.columns:
            preds = predict_time(df['t'].values)
            print('Predições em lote (primeiros 5):', preds[:5])
        else:
            print('CSV encontrado, mas sem coluna t')
    else:
        print('Nenhum CSV de inferência encontrado (data/times_for_infer.csv)')


### 5) Como persistir os artefatos no Google Drive

Se você executou este notebook no Colab e deseja salvar os artefatos (`model/` e `data/`) no seu Google Drive:

1. Monte o Drive (se ainda não montou):

```python
from google.colab import drive
drive.mount('/content/drive')
```

2. Copie os diretórios:

```python
import shutil
shutil.copy('model/fall_model.keras', '/content/drive/MyDrive/fall_model.keras')
shutil.copy('model/scaler.npz', '/content/drive/MyDrive/scaler.npz')
shutil.copy('data/dataset.npz', '/content/drive/MyDrive/dataset.npz')
shutil.copy('data/times_for_infer.csv', '/content/drive/MyDrive/times_for_infer.csv')
```

Ou copie a pasta inteira:

```python
!cp -r model /content/drive/MyDrive/
!cp -r data /content/drive/MyDrive/
```

Observação: se preferir, ajuste os caminhos de destino no seu Drive para organizar os arquivos em uma pasta de curso/aula.


Observação: a exportação para SavedModel é opcional. Use-a apenas se precisar do diretório `model/fall_model/` para TFSMLayer ou deploy.

---

Pronto. Execute as células de cima para baixo; se ocorrer qualquer erro, cole o traceback aqui e eu corrijo rapidamente.

In [ ]:
# Exportar SavedModel (opcional)
# Uma pasta SavedModel (`model/fall_model`) é necessária se você quiser usar TFSMLayer ou ferramentas que esperam o formato SavedModel.
# A célula abaixo tenta exportar automaticamente a partir do objeto Keras salvo: usa `model.export` (Keras 3) com fallback para `tf.saved_model.save`.
import os
import tensorflow as tf
export_dir = 'model/fall_model'
if os.path.exists(export_dir) and os.listdir(export_dir):
    print('SavedModel já existe em', export_dir)
else:
    try:
        # Keras 3: model.export cria SavedModel compatível
        print('Tentando exportar SavedModel com model.export(...)')
        model.export(export_dir)
        print('Exportado com model.export para', export_dir)
    except Exception as e:
        print('model.export não disponível ou falhou:', e)
        print('Tentando tf.saved_model.save como fallback...')
        try:
            tf.saved_model.save(model, export_dir)
            print('SavedModel salvo em', export_dir)
        except Exception as e2:
            print('Falha ao salvar SavedModel:', e2)
            print('Você ainda pode carregar o arquivo .keras com load_model("model/fall_model.keras").')
